* Ray RLlib 노트북

필요 패키지 삽입

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

from horcrux_terrain_v1.envs import SandWorld
from horcrux_terrain_v1.envs import PlaneWorld

import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig

from ray.tune.registry import register_env


c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Ray 실행 (Warning 관련 무시 키워드)

In [2]:
ray.init(runtime_env={"env_vars": {"PYTHONWARNINGS": "ignore::DeprecationWarning"}})

2024-09-24 01:40:39,864	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.5
Ray version:,2.36.0
Dashboard:,http://127.0.0.1:8265


Gym -> Rllib Env 등록

In [3]:
env_config = {
    "forward_reward_weight": 4,
    "side_cost_weight": 2,
    "unhealthy_max_steps": 75,
    "healthy_roll_range": (-45,45),
    "rotation_norm_cost_weight": 1,
    "termination_reward": 100,
}

# Sand
register_env("sand-v1", lambda config: SandWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 termination_reward=env_config["termination_reward"]))

# Plane
register_env("plane-v1", lambda config: PlaneWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 termination_reward=env_config["termination_reward"]))


학습 알고리즘 설정

In [4]:
config = PPOConfig()
# Activate new API stack.
config.api_stack(
    enable_rl_module_and_learner=True,
    enable_env_runner_and_connector_v2=True,
)
# config.environment("sand-v1")
config.environment("plane-v1")
config.framework("torch")
config.resources(num_gpus=1, num_cpus_for_main_process=12)

num_env_runners = 12
gpu_per_env_runner = float((1-0.05)/num_env_runners)
config.env_runners(num_env_runners=num_env_runners, num_gpus_per_env_runner=gpu_per_env_runner)
config.training(
    gamma=0.9, 
    lr=0.0001, 
    # kl_coeff=0.3, 
    train_batch_size_per_learner=96000,              
    sgd_minibatch_size=32768,

    # See model catalog for more options.
    # https://docs.ray.io/en/latest/rllib/rllib-models.html
    model={ "uses_new_env_runners": True,
            "fcnet_hiddens": [1024, 1024, 1024, 1024, 1024],
            "use_lstm": True,
            "lstm_cell_size": 4096,
            "max_seq_len": 200,
            "lstm_use_prev_action": True},
)
config.evaluation(evaluation_interval=100)

# # See model catalog for more options.
# # https://docs.ray.io/en/latest/rllib/rllib-models.html
# # config.model["fcnet_hiddens"] = [512, 512, 512, 512, 512]
# config.model["uses_new_env_runners"] = True
# config.model["fcnet_hiddens"] = [1024, 1024, 1024, 1024, 1024]
# config.model["use_lstm"] = True
# # config.model["lstm_cell_size"] = 2048
# config.model["lstm_cell_size"] = 4096
# config.model["max_seq_len"] = 200
# config.model["lstm_use_prev_action"] = True

algo = config.build()



c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\rllib\algorithms\algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be remove

혹시 이전 학습 결과를 로드할 경우

In [ ]:
algo.restore("./agents/1024x5_4096_New_lr001_LSTM_wGPU_final")

학습 파라미터 재조정

In [5]:
# algo.get_config().training().gamma

#Env runner 파라미터 보기.
algo.env_runner.config["exploration_config"]
algo.get_config().model

{'_disable_preprocessor_api': False,
 '_disable_action_flattening': False,
 'fcnet_hiddens': [1024, 1024, 1024, 1024, 1024],
 'fcnet_activation': 'tanh',
 'fcnet_weights_initializer': None,
 'fcnet_weights_initializer_config': None,
 'fcnet_bias_initializer': None,
 'fcnet_bias_initializer_config': None,
 'conv_filters': None,
 'conv_activation': 'relu',
 'conv_kernel_initializer': None,
 'conv_kernel_initializer_config': None,
 'conv_bias_initializer': None,
 'conv_bias_initializer_config': None,
 'conv_transpose_kernel_initializer': None,
 'conv_transpose_kernel_initializer_config': None,
 'conv_transpose_bias_initializer': None,
 'conv_transpose_bias_initializer_config': None,
 'post_fcnet_hiddens': [],
 'post_fcnet_activation': 'relu',
 'post_fcnet_weights_initializer': None,
 'post_fcnet_weights_initializer_config': None,
 'post_fcnet_bias_initializer': None,
 'post_fcnet_bias_initializer_config': None,
 'free_log_std': False,
 'no_final_linear': False,
 'vf_share_layers': False,


학습 시작

In [ ]:
from pprint import pprint

n_iter = 800
save_iter = 0
save_name = "1024x5_4096_New_lr0001_gamma08_LSTM_wGPU"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}th iteration done")
    # result.pop("config")
    # pprint(result)

    if i%60 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

algo.save(save_name+str("_final"))

환경에서 학습된 Policy 테스트하기

In [6]:
from ray.rllib.core.rl_module import RLModule
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
import time

rl_module = RLModule.from_checkpoint(
    pathlib.Path("./agents/1024x5_4096_New_lr0001_gamma08_LSTM_wGPU_final") / "learner_group" / "learner" / "rl_module"
)["default_policy"]

rl_module

PPOTorchRLModule(
  (encoder): TorchActorCriticEncoder(
    (actor_encoder): TorchMLPEncoder(
      (net): TorchMLP(
        (mlp): Sequential(
          (0): Linear(in_features=94, out_features=256, bias=True)
          (1): Tanh()
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Tanh()
        )
      )
    )
    (critic_encoder): TorchMLPEncoder(
      (net): TorchMLP(
        (mlp): Sequential(
          (0): Linear(in_features=94, out_features=256, bias=True)
          (1): Tanh()
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Tanh()
        )
      )
    )
  )
  (pi): TorchMLPHead(
    (net): TorchMLP(
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=28, bias=True)
      )
    )
  )
  (vf): TorchMLPHead(
    (net): TorchMLP(
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)

In [ ]:
# env = gym.make("horcrux_terrain_v1/sand-v1", 
#                terminate_when_unhealthy = False, 
#                render_mode = "human", 
#             #    render_camera_name = 'ceiling', 
#                use_gait = True,
#                gait_params = (30,30,40,40,0),
#                **env_config,
#                ) 

env = gym.make("horcrux_terrain_v1/plane-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 



for j in range(10):
   episode_return = 0
   terminated = truncated = False

   obs, info = env.reset()

   torch_obs_batch = torch.from_numpy(np.array([obs]))

   for i in range(1000):
      torch_obs_batch = torch.from_numpy(np.array([obs]))
      action_logits = rl_module.forward_inference({"obs": torch_obs_batch})[
         "action_dist_inputs"
      ]
      
      action = torch.argmax(action_logits[0]).numpy()

      obs, reward, terminated, truncated, info = env.step(action)

      if terminated:
         print("terminated")

      episode_return += reward

   print(f"Reached episode return of {episode_return}.")

env.close()